# Data processing for census sections, incomes, population and buildings

In the below data processing pipeline, the following datasets are used and generated:
1. **seccionado.geojson**: Data about census sections, downloaded from INE
2. **seccionado_procesado_EPSG_25830.geojson**: Subset of the previous dataset, containing only three fields ('CUSEC', 'area_m2', 'geometry')
3. **renta.csv**: Income dataset downloaded from INE
4. **renta_procesado_2020.csv**: Income by census section
5. **poblacion.csv**: Population dataset downloaded from INE
6. **poblacion_procesado_2020.csv**: Population by section
7. **renta_poblacion_secciones_2020.csv**: Income and Population for each section
8. **edificios_adecuados_EPSG_25830.geojson**: Buildings from Crevillent with Solar radiation data
9. **edificios_secciones_crevillente**: Sections, buildings and radiation data. This is the final dataset for querying in the backend.

**Note**: For building a final dataset for other region, solar radiation data for that area is necessary.


## Census sections
Data from 2023. Downloaded from: https://www.ine.es/ss/Satellite?L=es_ES&c=Page&cid=1259952026632&p=1259952026632&pagename=ProductosYServicios%2FPYSLayout

In [10]:
import dask_geopandas as dask_gpd
dgdf = dask_gpd.read_file('seccionado.geojson', npartitions=4) #EPSG:25830 (m2)

In [11]:
dgdf['area_m2'] = dgdf.area
dgdf = dgdf[['CUSEC', 'area_m2', 'geometry']]

In [12]:
computed_gdf = dgdf.compute()

In [13]:
computed_gdf.to_file('seccionado_procesado_EPSG_25830.geojson', driver='GeoJSON')

## Income by census sections
Data downloaded from: https://www.ine.es/dynt3/inebase/index.htm?padre=7132.

In [15]:
import dask.dataframe as dd
dtype_dict = {
    'Distritos': 'object',
    'Secciones': 'object',
    'Total': 'object'
}

In [16]:
df = dd.read_csv('renta.csv', dtype=dtype_dict, sep=';')

In [17]:
df = df[['Secciones', 'Indicadores de renta media', 'Periodo', 'Total']]

In [18]:
df = df[df['Indicadores de renta media'] == 'Renta neta media por hogar']

In [19]:
df = df[df['Periodo'] == 2020]

In [20]:
df = df[['Secciones', 'Periodo', 'Total']]

In [21]:
df = df.dropna()

In [22]:
df = df[['Secciones', 'Total']] # Periodo = 2020

In [23]:
df['Secciones'] = df['Secciones'].str.split().str[0]

In [24]:
df.to_csv('renta_procesado_2020_parts.csv', index=False)

['C:\\Users\\omar.alvarez\\Downloads\\data\\seccionado_2023\\Seccionado2023\\Seccionado2023\\España_Seccionado2023_ETRS89H30\\renta_procesado_2020_parts.csv\\0.part',
 'C:\\Users\\omar.alvarez\\Downloads\\data\\seccionado_2023\\Seccionado2023\\Seccionado2023\\España_Seccionado2023_ETRS89H30\\renta_procesado_2020_parts.csv\\1.part',
 'C:\\Users\\omar.alvarez\\Downloads\\data\\seccionado_2023\\Seccionado2023\\Seccionado2023\\España_Seccionado2023_ETRS89H30\\renta_procesado_2020_parts.csv\\2.part']

In [25]:
filenames = "renta_procesado_2020_parts.csv/*.part"
dtype_dict = {
    'Secciones': 'object',
    'Total': 'object'
}
ddf = dd.read_csv(filenames, dtype=dtype_dict, sep=',')

In [26]:
ddf.compute().to_csv('renta_procesado_2020.csv', index=False)

## Population

Data downloaded from: https://www.ine.es/jaxiT3/Tabla.htm?t=30868&L=0

In [27]:
import dask.dataframe as dd
dtype_dict = {
    'Secciones': 'object',
    'Indicadores demográficos': 'object'

}
df = dd.read_csv('poblacion.csv', dtype=dtype_dict, sep=';')

In [28]:
df = df[df['Periodo'] == 2020]

In [29]:
df = df[['Secciones', 'Indicadores demográficos', 'Total']]

In [30]:
df['Secciones'] = df['Secciones'].str.split().str[0]

In [31]:
df['Indicadores demográficos'] = df['Indicadores demográficos'].astype('category')

In [32]:
df['Indicadores demográficos'] = df['Indicadores demográficos'].cat.as_known()

In [33]:
df = df.dropna()

In [34]:
df_edad_media = df[df['Indicadores demográficos'] == 'Edad media de la población']
df_poblacion = df[df['Indicadores demográficos'] == 'Población']
df_porcentaje_unipersonales = df[df['Indicadores demográficos'] == 'Porcentaje de hogares unipersonales']
df_porcentaje_mayor_65 = df[df['Indicadores demográficos'] == 'Porcentaje de población de 65 y más años']
df_porcentaje_menor_18 = df[df['Indicadores demográficos'] == 'Porcentaje de población menor de 18 años']
df_porcentaje_poblacion = df[df['Indicadores demográficos'] == 'Porcentaje de población española']
df_tamaño_medio = df[df['Indicadores demográficos'] == 'Tamaño medio del hogar']

In [35]:
df_dataset = dd.merge(df_edad_media, df_poblacion, on='Secciones', how='inner', suffixes=('_edad_media', '_poblacion'))
df_dataset = df_dataset[['Secciones', 'Total_edad_media', 'Total_poblacion']]
df_dataset = dd.merge(df_dataset, df_porcentaje_unipersonales, on='Secciones', how='inner', suffixes=('_%_unipersonales', '_%_unipersonales'))
df_dataset = df_dataset.rename(columns={'Total': 'Porcentaje_Hogares_unipersonales'})
df_dataset = df_dataset.drop(columns=['Indicadores demográficos'])
df_dataset = dd.merge(df_dataset, df_porcentaje_mayor_65, on='Secciones', how='inner')
df_dataset = df_dataset.rename(columns={'Total': 'Porcentaje_mayor_65'})
df_dataset = df_dataset.drop(columns=['Indicadores demográficos'])
df_dataset = dd.merge(df_dataset, df_porcentaje_menor_18, on='Secciones', how='inner')
df_dataset = df_dataset.rename(columns={'Total': 'Porcentaje_menor_18'})
df_dataset = df_dataset.drop(columns=['Indicadores demográficos'])
df_dataset = dd.merge(df_dataset, df_porcentaje_poblacion, on='Secciones', how='inner')
df_dataset = df_dataset.rename(columns={'Total': 'Porcentaje_poblacion'})
df_dataset = df_dataset.drop(columns=['Indicadores demográficos'])
df_dataset = dd.merge(df_dataset, df_tamaño_medio, on='Secciones', how='inner')
df_dataset = df_dataset.rename(columns={'Total': 'Tamaño_medio_hogar'})
df_dataset = df_dataset.drop(columns=['Indicadores demográficos'])

In [36]:
df_dataset.to_csv('poblacion_procesado_2020_parts.csv', index=False)

['C:\\Users\\omar.alvarez\\Downloads\\data\\seccionado_2023\\Seccionado2023\\Seccionado2023\\España_Seccionado2023_ETRS89H30\\poblacion_procesado_2020_parts.csv\\0.part',
 'C:\\Users\\omar.alvarez\\Downloads\\data\\seccionado_2023\\Seccionado2023\\Seccionado2023\\España_Seccionado2023_ETRS89H30\\poblacion_procesado_2020_parts.csv\\1.part',
 'C:\\Users\\omar.alvarez\\Downloads\\data\\seccionado_2023\\Seccionado2023\\Seccionado2023\\España_Seccionado2023_ETRS89H30\\poblacion_procesado_2020_parts.csv\\2.part',
 'C:\\Users\\omar.alvarez\\Downloads\\data\\seccionado_2023\\Seccionado2023\\Seccionado2023\\España_Seccionado2023_ETRS89H30\\poblacion_procesado_2020_parts.csv\\3.part']

In [37]:
filenames = "poblacion_procesado_2020_parts.csv/*.part"
dtype_dict = {
    'Secciones': 'object',
}
ddf = dd.read_csv(filenames, dtype=dtype_dict, sep=',')

In [38]:
ddf.compute().to_csv('poblacion_procesado_2020.csv', index=False)

## Data fusion: population, income and sections

In [39]:
import dask.dataframe as dd
dtype_dict = {
    'Secciones': 'object',
    'Total': 'object'

}

df_renta = dd.read_csv('renta_procesado_2020.csv', dtype=dtype_dict, sep=',')

In [40]:
df_renta = df_renta.rename(columns={"Total":"Renta_media"})

dtype_dict = {
    'Secciones': 'object',

}

In [41]:
df_poblacion = dd.read_csv('poblacion_procesado_2020.csv', dtype=dtype_dict, sep=',')

In [42]:
df_renta_poblacion = df_renta.merge(df_poblacion, on='Secciones', how='inner').compute()

### Joining with sections

In [67]:
import dask_geopandas as dg
df_seccionado = dg.read_file('seccionado_procesado_EPSG_25830.geojson', npartitions=4)
df_seccionado = df_seccionado.rename(columns={"CUSEC":"Secciones"})


In [68]:
import geopandas as gpd
# df_renta_poblacion_seccion = df_renta_poblacion.merge(df_seccionado, on='Secciones', how='inner')
df_renta_poblacion_seccion = gpd.GeoDataFrame(df_renta_poblacion).merge(df_seccionado.compute(), on='Secciones', how='inner')


In [69]:
dd_renta_poblacion_seccion = dd.from_pandas(df_renta_poblacion_seccion, npartitions=4)

In [70]:
dd_renta_poblacion_seccion.to_csv('renta_poblacion_secciones_2020_parts.csv', index=False)

['C:\\Users\\omar.alvarez\\Downloads\\data\\seccionado_2023\\Seccionado2023\\Seccionado2023\\España_Seccionado2023_ETRS89H30\\renta_poblacion_secciones_2020_parts.csv\\0.part',
 'C:\\Users\\omar.alvarez\\Downloads\\data\\seccionado_2023\\Seccionado2023\\Seccionado2023\\España_Seccionado2023_ETRS89H30\\renta_poblacion_secciones_2020_parts.csv\\1.part',
 'C:\\Users\\omar.alvarez\\Downloads\\data\\seccionado_2023\\Seccionado2023\\Seccionado2023\\España_Seccionado2023_ETRS89H30\\renta_poblacion_secciones_2020_parts.csv\\2.part',
 'C:\\Users\\omar.alvarez\\Downloads\\data\\seccionado_2023\\Seccionado2023\\Seccionado2023\\España_Seccionado2023_ETRS89H30\\renta_poblacion_secciones_2020_parts.csv\\3.part']

In [74]:
filenames = "renta_poblacion_secciones_2020_parts.csv/*.part"
dtype_dict = {
    'Secciones': 'object',
    'Renta_media': 'object',
    'Total_edad' : 'object',
    'area_m2': 'object'
}
ddf = dd.read_csv(filenames, dtype=dtype_dict, sep=',', thousands='.')

In [77]:
%who

computed_gdf	 dask_gpd	 dd	 dd_renta_poblacion_seccion	 ddf	 df	 df_dataset	 df_edad_media	 df_poblacion	 
df_porcentaje_mayor_65	 df_porcentaje_menor_18	 df_porcentaje_poblacion	 df_porcentaje_unipersonales	 df_renta	 df_renta_poblacion	 df_renta_poblacion_seccion	 df_seccionado	 df_tamaño_medio	 
dg	 dgdf	 dtype_dict	 filenames	 gpd	 


In [87]:
# To free RAM memory
import gc
del computed_gdf
del dask_gpd
del df_dataset
del df_edad_media
del df_poblacion
del df_porcentaje_mayor_65
del df_porcentaje_menor_18
del df_porcentaje_poblacion
del df_porcentaje_unipersonales
del df_renta
del df_renta_poblacion
del df_renta_poblacion_seccion
del df_seccionado
del df_tamaño_medio
gc.collect()

NameError: name 'computed_gdf' is not defined

In [88]:
ddf.compute().to_csv('renta_poblacion_secciones_2020.csv', index=False)

## Data fusion: Solar cadaster

In [90]:
%who

dd	 dd_renta_poblacion_seccion	 ddf	 df	 dg	 dgdf	 dtype_dict	 filenames	 gc	 
gpd	 


In [91]:
del dd_renta_poblacion_seccion
del ddf
del df
del dgdf
gc.collect()

779

In [92]:
import pandas as pd
df_secciones = pd.read_csv('renta_poblacion_secciones_2020.csv')

In [93]:
import geopandas as gpd
from shapely import wkt

df_secciones['geometry'] = df_secciones['geometry'].apply(wkt.loads)
gdf_secciones = gpd.GeoDataFrame(df_secciones, geometry='geometry')
gdf_secciones.crs = 'EPSG:25830'

To load only data from **Crevillent cadaster**:

In [94]:
gdf_edificios_adecuados = gpd.read_file('edificios_adecuados_EPSG_25830.geojson')
gdf_edificios_adecuados = gdf_edificios_adecuados[['reference', 'AREA', 'MEAN','MWh_aprove', 'MWh_prod_e', 'geometry' ]]

C:\Users\omar.alvarez\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\geopandas\io\file.py:383: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  as_dt = pd.to_datetime(df[k], errors="ignore")
C:\Users\omar.alvarez\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\geopandas\io\file.py:387: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  as_dt = pd.to_datetime(df[k], errors="ignore", utc=True)


In [95]:
gdf_edificios_adecuados['centroid'] = gdf_edificios_adecuados.geometry.centroid
gdf_edificios_adecuados = gdf_edificios_adecuados.set_geometry('centroid')

In [99]:

gdf_edificios_secciones = gpd.sjoin(gdf_edificios_adecuados, gdf_secciones, how="inner", predicate="within")
gdf_edificios_secciones = gdf_edificios_secciones.drop('index_right', axis=1)

In [100]:
secciones_no_crevillent = [300502002, 305903003, 390401001, 306507023]
gdf_edificios_secciones = gdf_edificios_secciones[~gdf_edificios_secciones['Secciones'].isin(secciones_no_crevillent)]

In [98]:
gdf_edificios_secciones.to_csv('edificios_secciones_crevillente.csv', index=False)